In [4]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [102]:
conn=psycopg2.connect("dbname='InvoiceData' user='postgres' password='asdf1234' port='5432' host='localhost'")
cur=conn.cursor()
cur.execute("CREATE TABLE InvoiceInfo(InvoiceDate TEXT, VehicleNo TEXT, InvoiceNo TEXT PRIMARY KEY, ReName TEXT, ReAddr TEXT, ReState TEXT, ReGST TEXT, RePAN TEXT, CeName TEXT, CeAddr TEXT, CeState TEXT, CeGST TEXT, CePAN TEXT, Items INTEGER )")
cur.execute("CREATE TABLE InvoiceContent(InvoiceNo TEXT REFERENCES InvoiceInfo(InvoiceNo),Description TEXT, HSN TEXT, Qty TEXT, Rate TEXT, CGST REAL, SGST REAL, IGST REAL, Total REAL)")
conn.commit()
conn.close()


In [95]:
conn=psycopg2.connect("dbname='InvoiceData' user='postgres' password='asdf1234' port='5432' host='localhost'")
cur=conn.cursor()
cur.execute("CREATE TABLE SellerInfo(InvoiceDate TEXT, VehicleNo TEXT, InvoiceNo TEXT PRIMARY KEY, ReName TEXT, ReAddr TEXT, ReState TEXT, ReGST TEXT, RePAN TEXT, Items INTEGER )")
cur.execute("CREATE TABLE SellerContent(InvoiceNo TEXT REFERENCES SellerInfo(InvoiceNo),Description TEXT, HSN TEXT, Qty TEXT, Rate TEXT, CGST REAL, SGST REAL, IGST REAL, Total REAL)")
conn.commit()
conn.close()

In [96]:
conn=psycopg2.connect("dbname='InvoiceData' user='postgres' password='asdf1234' port='5432' host='localhost'")
cur=conn.cursor()
cur.execute("CREATE TABLE BalanceInfo(InvoiceDate TEXT, VehicleNo TEXT, InvoiceNo TEXT PRIMARY KEY, ReName TEXT, ReAddr TEXT, ReState TEXT, ReGST TEXT, RePAN TEXT, Items INTEGER )")
cur.execute("CREATE TABLE BalanceContent(InvoiceNo TEXT REFERENCES BalanceInfo(InvoiceNo),Description TEXT, HSN TEXT, Qty TEXT, Rate TEXT, CGST REAL, SGST REAL, IGST REAL, Total REAL)")
conn.commit()
conn.close()

In [1]:
from tkinter import *
from tkcalendar import DateEntry
from tkinter import ttk
from functools import partial
import psycopg2
from copy import deepcopy

In [2]:
def sale(start):
    win=Toplevel(start)
    Invoiceno=StringVar()
    InvoiceDate=StringVar()
    Vehicle=StringVar()
    win.geometry("800x600")
    win.resizable(0,0)
    head=Label(win,text='TAX INVOICE',font="Arial 20")
    head.place(x=300,y=10)
    l1=Label(win,text='Invoice Date')
    l1.place(x=30,y=70)

    cal = DateEntry(win, width=17, background='black',foreground='white', borderwidth=2)
    cal.place(x=110,y=70)
    InvoiceDate=0
    #print(InvoiceDate)

    #e1=Entry(win,textvariable=BillNo)
    #e1.place(x=100,y=50)
    l2=Label(win,text="Invoice No")
    l2.place(x=510,y=70)
    e2=Entry(win,textvariable=Invoiceno)
    e2.place(x=590,y=70)

    l3=Label(win,text="Vehicle No")
    l3.place(x=270,y=70)
    e3=Entry(win,textvariable=Vehicle)
    e3.place(x=350,y=70)
    ttk.Separator(win,orient='horizontal').place(y=100,width=800,height=2)
    #---------------------------------------------------------------------------------------------------------------------------
    p1=PanedWindow(win,bg='#F8F9F9',height=200,width=360)
    p1.place(x=20,y=120)
    ttk.Separator(win,orient='vertical').place(x=396,y=120,width=10,height=200)
    p2=PanedWindow(win,bg='#F8F9F9',height=200,width=360)
    p2.place(x=415,y=120)
    #---------------------------------------------------------------------------------------------------------------------------
    Label(p1,text="Receiver Details (Billed To):",bg="#F8F9F9",font='Arial 10 underline').place(relx=0.02,rely=0.02)
    rename=StringVar()
    reAddr=StringVar()
    reState=StringVar()
    reState.set("Punjab")
    reGST=StringVar()
    rePAN=StringVar()
    options=['Andhra Pradesh', 'Arunachal Pradesh', 'Assam', 'Bihar', 'Chhattisgarh', 'Goa', 'Gujarat', 'Haryana', 'Himachal Pradesh', 'Jharkhand', 'Karnataka', 'Kerala','Madhya Pradesh', 'Maharashtra', 'Manipur', 'Meghalaya', 'Mizoram', 'Nagaland', 'Odisha', 'Punjab', 'Rajasthan', 'Sikkim', 'Tamil Nadu', 'Telangana', 'Tripura', 'Uttar Pradesh', 'Uttarakhand', 'West Bengal']

    Label(p1,text='Name',bg="#F8F9F9").place(relx=0.02,rely=0.2)
    Entry(p1,textvariable=rename).place(relx=0.25,rely=0.2,width=100)

    Label(p1,text='Address',bg="#F8F9F9").place(relx=0.02,rely=0.35)
    Entry(p1,textvariable=reAddr).place(relx=0.25,rely=0.35,width=200)

    Label(p1,text='State',bg="#F8F9F9").place(relx=0.02,rely=0.5)
    OptionMenu( p1 , reState , *options).place(relx=0.25,rely=0.5,height=25,width=150) 

    Label(p1,text='GSTIN',bg="#F8F9F9").place(relx=0.02,rely=0.65)
    Entry(p1,textvariable=reGST).place(relx=0.25,rely=0.65,width=100)

    Label(p1,text='PAN',bg="#F8F9F9").place(relx=0.02,rely=0.8)
    Entry(p1,textvariable=rePAN).place(relx=0.25,rely=0.8,width=100)
    opt=IntVar()
    def same(ee1,ee2,ee3,ee4,o1,e1,e2,e3,e4,o):
        ee1.insert(0,e1.get())
        ee2.insert(0,e2.get())
        ee3.insert(0,e3.get())
        ee4.insert(0,e4.get())
        o1.set(o.get())
    #---------------------------------------------------------------------------------------------------------------------------
    Label(p2,text="Consignee Details (Shipped To):",bg="#F8F9F9",font='Arial 10 underline').place(relx=0.02,rely=0.02)
    cename=StringVar()
    ceAddr=StringVar()
    ceState=StringVar()
    ceState.set("Punjab")
    ceGST=StringVar()
    cePAN=StringVar()    
    options=['Andhra Pradesh', 'Arunachal Pradesh', 'Assam', 'Bihar', 'Chhattisgarh', 'Goa', 'Gujarat', 'Haryana', 'Himachal Pradesh', 'Jharkhand', 'Karnataka', 'Kerala','Madhya Pradesh', 'Maharashtra', 'Manipur', 'Meghalaya', 'Mizoram', 'Nagaland', 'Odisha', 'Punjab', 'Rajasthan', 'Sikkim', 'Tamil Nadu', 'Telangana', 'Tripura', 'Uttar Pradesh', 'Uttarakhand', 'West Bengal']

    Label(p2,text='Name',bg="#F8F9F9").place(relx=0.02,rely=0.2)
    ee1=Entry(p2,textvariable=cename)
    ee1.place(relx=0.25,rely=0.2,width=100)

    Label(p2,text='Address',bg="#F8F9F9").place(relx=0.02,rely=0.35)
    ee2=Entry(p2,textvariable=ceAddr)
    ee2.place(relx=0.25,rely=0.35,width=200)

    Label(p2,text='State',bg="#F8F9F9").place(relx=0.02,rely=0.5)
    o1=OptionMenu( p2 , ceState , *options)
    o1.place(relx=0.25,rely=0.5,height=25,width=150) 

    Label(p2,text='GSTIN',bg="#F8F9F9").place(relx=0.02,rely=0.65)
    ee3=Entry(p2,textvariable=ceGST)
    ee3.place(relx=0.25,rely=0.65,width=100)

    Label(p2,text='PAN',bg="#F8F9F9").place(relx=0.02,rely=0.8)
    ee4=Entry(p2,textvariable=cePAN)
    ee4.place(relx=0.25,rely=0.8,width=100)
    same=partial(same,ee1,ee2,ee3,ee4,ceState,rename,reAddr,reGST,rePAN,reState)
    Button(p1,text="Shipping Address same as Billing Address",command=same,bg="#F8F9F9").place(relx=0.25,rely=0.9)

    #----------------------------------------------------------------------------------------------------------------------------
    ttk.Separator(win,orient='horizontal').place(y=340,width=800,height=2)
    p3=PanedWindow(win,bg='#F8F9F9',height=200,width=760)
    p3.place(x=20,y=360)
    itms=StringVar()
    def mycommand(itms,p,par,state,idate,ino,veh,rname,raddr,rstate,rgst,rpan,cname,caddr,cegst,cpan):
        for widget in p.winfo_children():
            widget.destroy()
        idate=cal.get_date()
        Label(p,text="Sno",bg="#F8F9F9").place(relx=0.02,rely=0.15)
        Label(p,text="Description",bg="#F8F9F9").place(relx=0.08,rely=0.15)
        Label(p,text="HSN/SAC",bg="#F8F9F9").place(relx=0.3,rely=0.15)
        Label(p,text="Qty (in cmt)",bg="#F8F9F9").place(relx=0.4,rely=0.15)
        Label(p,text="Rate",bg="#F8F9F9").place(relx=0.5,rely=0.15)
        Label(p,text="CGST",bg="#F8F9F9").place(relx=0.6,rely=0.15)
        Label(p,text="SGST",bg="#F8F9F9").place(relx=0.7,rely=0.15)
        Label(p,text="IGST",bg="#F8F9F9").place(relx=0.8,rely=0.15)
        Label(p,text="Total",bg="#F8F9F9").place(relx=0.9,rely=0.15)
        def calc(opts,qty,rate,e,x,state,p,idate,ino,veh,rname,raddr,rstate,rgst,rpan,cname,caddr,cegst,cpan):
            idate=cal.get_date()
            for i in range(x):
                conn=psycopg2.connect("dbname='InvoiceData' user='postgres' password='asdf1234' port='5432' host='localhost'")
                cur=conn.cursor()
                cur.execute("SELECT * FROM BalanceContent WHERE Description= (%s)",(str(opts[i].get()),))
                info=cur.fetchall()
                conn.commit()
                conn.close()
                s1=0
                for j in info:
                    s1=s1+int(j[3])
                if int(qty[i].get())>s1:
                    for j in p.winfo_children():
                        j.destroy()
                    Label(p,text="Not Enough Stock").place(relx=0.6,rely=0.15)
                    break
            else:
                
                for i in range(x):
                    dummy=deepcopy(int(qty[i].get()))
                    conn=psycopg2.connect("dbname='InvoiceData' user='postgres' password='asdf1234' port='5432' host='localhost'")
                    cur=conn.cursor()
                    cur.execute("SELECT * FROM BalanceContent WHERE Description= (%s)",(str(opts[i].get()),))
                    info=cur.fetchall()
                    conn.commit()
                    conn.close()
                    s1=0
                    for j in range(len(info)):
                        info[j]=list(info[j])
                        if dummy-int(info[j][3])>0:
                            dummy=dummy-int(info[j][3])
                            info[j][3]='0'
                        else:
                            info[j][3]=str(int(info[j][3])-dummy)
                            dummy=0
                    conn=psycopg2.connect("dbname='InvoiceData' user='postgres' password='asdf1234' port='5432' host='localhost'")
                    cur=conn.cursor()
                    for j in info:
                        cur.execute("UPDATE BalanceContent SET Qty=(%s) WHERE Description= (%s) AND InvoiceNo=(%s)",(j[3],j[1],j[0]))
                    conn.commit()
                    conn.close()
                hsn={"Teak Wood":"44","Imported Wood":"4407","Babule(Kukkar)":"4407","Firewood":"-"}
                cgst=[]
                sgst=[]
                igst=[]
                total=[]
                subtotal=[]
                for i in range(x):
                    subtotal.append(round(float(qty[i].get())*float(rate[i].get()),2))
                    if opts[i].get()!="Firewood":
                        if state.get()=="Punjab":
                            cgst.append(round(subtotal[i]*9/100,2))
                            sgst.append(round(subtotal[i]*9/100,2))
                            igst.append(0)
                        else:
                            cgst.append(0)
                            sgst.append(0)
                            igst.append(round(subtotal[i]*18/100,2))
                    else:
                        cgst.append(0)
                        sgst.append(0)
                        igst.append(0)
                    total.append(round(subtotal[i]+cgst[i]+sgst[i]+igst[i],2))
                    Label(p,text="Rs."+str(cgst[i]),bg="#F8F9F9").place(relx=0.6,rely=0.15*(i+2))
                    Label(p,text="Rs."+str(sgst[i]),bg="#F8F9F9").place(relx=0.7,rely=0.15*(i+2))
                    Label(p,text="Rs."+str(igst[i]),bg="#F8F9F9").place(relx=0.8,rely=0.15*(i+2))
                    Label(p,text="Rs."+str(total[i]),bg="#F8F9F9").place(relx=0.9,rely=0.15*(i+2))
                    e[i].insert(0,hsn[opts[i].get()])
                grand=round(sum(total),2)
                Label(p,text="Grand Total: Rs."+str(grand),bg="#F8F9F9").place(relx=0.75,rely=0.90)
                conn=psycopg2.connect("dbname='InvoiceData' user='postgres' password='asdf1234' port='5432' host='localhost'")
                cur=conn.cursor()
                cur.execute(" INSERT INTO InvoiceInfo VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)",(str(idate),str(veh.get()),str(ino.get()),str(rname.get()),str(raddr.get()),str(rstate.get()),str(rgst.get()),str(rpan.get()),str(cname.get()),str(caddr.get()),str(state.get()),str(cegst.get()),str(cpan.get()),str(x)))
                for i in range(x):
                    cur.execute(" INSERT INTO InvoiceContent VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s)",(str(ino.get()),str(opts[i].get()),hsn[opts[i].get()],str(qty[i].get()),str(rate[i].get()),str(cgst[i]),str(sgst[i]),str(igst[i]),str(total[i])))
                conn.commit()
                conn.close()
        x=int(itms.get())
        ops=["Teak Wood","Imported Wood","Babule(Kukkar)","Firewood"]
        qty=[]
        opts=[]
        e=[]
        rate=[]
        for i in range(x):
            opts.append(StringVar())
            qty.append(StringVar())
            rate.append(StringVar())

        for i in range(x):
            Label(p,text=str(i+1),bg="#F8F9F9").place(relx=0.02,rely=0.15*(i+2))
            OptionMenu( p , opts[i] , *ops).place(relx=0.08,rely=0.15*(i+2),height=25,width=150)
            e.append(Entry(p))
            e[i].place(relx=0.3,rely=0.15*(i+2),height=25,width=50)
            Entry(p,textvariable=qty[i]).place(relx=0.4,rely=0.15*(i+2),height=25,width=50)
            Entry(p,textvariable=rate[i]).place(relx=0.5,rely=0.15*(i+2),height=25,width=50)
        calc=partial(calc,opts,qty,rate,e,x,state,p,idate,ino,veh,rname,raddr,rstate,rgst,rpan,cname,caddr,cegst,cpan)    
        Button(par,text="Calculate",command=calc).place(relx=0.7,rely=0.05,height=20,width=90)


    p4=PanedWindow(win,bg='#F8F9F9',height=190,width=760)
    p4.place(x=20,y=390)        
    mycommand=partial(mycommand,itms,p4,p3,ceState,InvoiceDate,Invoiceno,Vehicle,rename,reAddr,reState,reGST,rePAN,cename,ceAddr,ceGST,cePAN)
    Label(p3,text='No of items',bg="#F8F9F9").place(relx=0.02,rely=0.05)
    Spinbox(p3,textvariable=itms,from_=1,to=4).place(relx=0.15,rely=0.05,width=100)
    Button(p3,text="Generate",command=mycommand).place(relx=0.32,rely=0.05,height=20,width=90)
    def srch(win):
        tp=Toplevel(win)
        tp.geometry("600x550")
        tp.resizable(0,0)
        Label(tp,text="Enter the Invoice No.").place(relx=0.1,rely=0.1)
        invno=StringVar()
        Entry(tp,textvariable=invno).place(relx=0.35,rely=0.1)
        def select(invno,pan):
            for widget in pan.winfo_children():
                widget.destroy()

            conn=psycopg2.connect("dbname='InvoiceData' user='postgres' password='asdf1234' port='5432' host='localhost'")
            cur=conn.cursor()
            cur.execute("SELECT * FROM InvoiceInfo WHERE InvoiceNo= (%s)",(str(invno.get()),))
            info=cur.fetchall()
            conn.commit()
            conn.close()
            if len(info)==0:
                Label(pan,text="No Info Found",bg="#F8F9F9",font="Arial 15").place(relx=0.02,rely=0.08)
            else:
                Label(pan,text="TAX INVOICE",bg='#F8F9F9',justify="center",font="Arial 15").place(relx=0.35,rely=0.09)
                ttk.Separator(pan,orient='horizontal').place(relx=0,rely=0.17,height=2,width=500)
                ttk.Separator(pan,orient='horizontal').place(relx=0,rely=0.6,height=2,width=500)
                s="Invoice Date: "+str(info[0][0])+"\tVehicle No: "+str(info[0][1])+"\tInvoice No: "+str(info[0][2])
                s1="Receiver Information:\nName: "+str(info[0][3])+"\nAddress: "+str(" ".join(info[0][4].split(" ")[:4]))+"\n"+str(" ".join(info[0][4].split(" ")[4:8]))+"\n"+str(" ".join(info[0][4].split(" ")[8:]))+"\nState: "+str(info[0][5])+"\nGSTIN: "+str(info[0][6])+"\nPAN: "+str(info[0][7])
                s2="Consignee Information:"+"\nName: "+str(info[0][8])+"\nAddress: "+str(" ".join(info[0][4].split(" ")[:4]))+"\n"+str(" ".join(info[0][4].split(" ")[4:8]))+"\n"+str(" ".join(info[0][4].split(" ")[8:]))+"\nState: "+str(info[0][10])+"\nGSTIN: "+str(info[0][11])+"\nPAN: "+str(info[0][12])
                Label(pan,text=s,bg='#F8F9F9',justify="left").place(relx=0.15,rely=0.2)
                Label(pan,text=s1,bg='#F8F9F9',justify="left").place(relx=0.15,rely=0.25)
                Label(pan,text=s2,bg='#F8F9F9',justify="left").place(relx=0.55,rely=0.25)
                conn=psycopg2.connect("dbname='InvoiceData' user='postgres' password='asdf1234' port='5432' host='localhost'")
                cur=conn.cursor()
                cur.execute("SELECT * FROM InvoiceContent WHERE InvoiceNo= (%s)",(str(invno.get()),))
                info=cur.fetchall()
                conn.commit()
                conn.close()
                gtot=0.0
                Label(pan,text="Sno",bg="#F8F9F9").place(relx=0.02,rely=0.63)
                Label(pan,text="Description",bg="#F8F9F9").place(relx=0.08,rely=0.63)
                Label(pan,text="HSN",bg="#F8F9F9").place(relx=0.3,rely=0.63)
                Label(pan,text="Qty \n(in cmt)",bg="#F8F9F9").place(relx=0.39,rely=0.63)
                Label(pan,text="Rate",bg="#F8F9F9").place(relx=0.5,rely=0.63)
                Label(pan,text="CGST",bg="#F8F9F9").place(relx=0.57,rely=0.63)
                Label(pan,text="SGST",bg="#F8F9F9").place(relx=0.68,rely=0.63)
                Label(pan,text="IGST",bg="#F8F9F9").place(relx=0.8,rely=0.63)
                Label(pan,text="Total",bg="#F8F9F9").place(relx=0.89,rely=0.63)
                for i in range(len(info)):
                    Label(pan,text=str(i+1),bg="#F8F9F9").place(relx=0.02,rely=0.63+(0.05*(i+2)))
                    Label(pan,text=str(info[i][1]),bg="#F8F9F9").place(relx=0.08,rely=0.63+(0.05*(i+2)))
                    Label(pan,text=str(info[i][2]),bg="#F8F9F9").place(relx=0.3,rely=0.63+(0.05*(i+2)))
                    Label(pan,text=str(info[i][3]),bg="#F8F9F9").place(relx=0.39,rely=0.63+(0.05*(i+2)))
                    Label(pan,text=str(info[i][4]),bg="#F8F9F9").place(relx=0.5,rely=0.63+(0.05*(i+2)))
                    Label(pan,text=str(info[i][5]),bg="#F8F9F9").place(relx=0.57,rely=0.63+(0.05*(i+2)))
                    Label(pan,text=str(info[i][6]),bg="#F8F9F9").place(relx=0.68,rely=0.63+(0.05*(i+2)))
                    Label(pan,text=str(info[i][7]),bg="#F8F9F9").place(relx=0.8,rely=0.63+(0.05*(i+2)))
                    Label(pan,text=str(info[i][8]),bg="#F8F9F9").place(relx=0.89,rely=0.63+(0.05*(i+2)))
                    gtot=gtot+float(info[i][8])
                Label(pan,text="Grand Total: Rs."+str(gtot),bg="#F8F9F9").place(relx=0.7,rely=0.95)
        pan=PanedWindow(tp,bg='#F8F9F9',height=400,width=500)
        pan.place(relx=0.08,rely=0.25)
        select=partial(select,invno,pan)
        Button(tp,text="Search",command=select).place(relx=0.7,rely=0.1,height=20,width=70)
    Button(win,text="Exit ",command=win.destroy).place(x=750,y=17)

    srch=partial(srch,win)
    Button(win,text="Search Invoice",command=srch).place(x=650,y=17)
    #l4=Label(win,text="Vehicle No")
    #l4.place(x=270,y=50)
    #e4=Entry(win,textvariable=GSTIN)
    #e4.place(x=350,y=50)
    



In [3]:
def purchase(start):
    win=Toplevel(start)
    Invoiceno=StringVar()
    InvoiceDate=StringVar()
    Vehicle=StringVar()
    win.geometry("800x600")
    win.resizable(0,0)
    head=Label(win,text='TAX INVOICE',font="Arial 20")
    head.place(x=300,y=10)
    l1=Label(win,text='Invoice Date')
    l1.place(x=30,y=70)

    cal = DateEntry(win, width=17, background='black',foreground='white', borderwidth=2)
    cal.place(x=110,y=70)
    InvoiceDate=0
    #print(InvoiceDate)

    #e1=Entry(win,textvariable=BillNo)
    #e1.place(x=100,y=50)
    l2=Label(win,text="Invoice No")
    l2.place(x=510,y=70)
    e2=Entry(win,textvariable=Invoiceno)
    e2.place(x=590,y=70)

    l3=Label(win,text="Vehicle No")
    l3.place(x=270,y=70)
    e3=Entry(win,textvariable=Vehicle)
    e3.place(x=350,y=70)
    ttk.Separator(win,orient='horizontal').place(y=100,width=800,height=2)
    #---------------------------------------------------------------------------------------------------------------------------
    p1=PanedWindow(win,bg='#F8F9F9',height=200,width=720)
    p1.place(x=20,y=120)
    #---------------------------------------------------------------------------------------------------------------------------
    Label(p1,text="Buyer:",bg="#F8F9F9",font='Arial 10 underline').place(relx=0.02,rely=0.02)
    rename=StringVar()
    reAddr=StringVar()
    reState=StringVar()
    reState.set("Punjab")
    reGST=StringVar()
    rePAN=StringVar()
    options=['Andhra Pradesh', 'Arunachal Pradesh', 'Assam', 'Bihar', 'Chhattisgarh', 'Goa', 'Gujarat', 'Haryana', 'Himachal Pradesh', 'Jharkhand', 'Karnataka', 'Kerala','Madhya Pradesh', 'Maharashtra', 'Manipur', 'Meghalaya', 'Mizoram', 'Nagaland', 'Odisha', 'Punjab', 'Rajasthan', 'Sikkim', 'Tamil Nadu', 'Telangana', 'Tripura', 'Uttar Pradesh', 'Uttarakhand', 'West Bengal']

    Label(p1,text='Name',bg="#F8F9F9").place(relx=0.02,rely=0.2)
    Entry(p1,textvariable=rename).place(relx=0.25,rely=0.2,width=100)

    Label(p1,text='Address',bg="#F8F9F9").place(relx=0.02,rely=0.35)
    Entry(p1,textvariable=reAddr).place(relx=0.25,rely=0.35,width=200)

    Label(p1,text='State',bg="#F8F9F9").place(relx=0.02,rely=0.5)
    OptionMenu( p1 , reState , *options).place(relx=0.25,rely=0.5,height=25,width=150) 

    Label(p1,text='GSTIN',bg="#F8F9F9").place(relx=0.02,rely=0.65)
    Entry(p1,textvariable=reGST).place(relx=0.25,rely=0.65,width=100)

    Label(p1,text='PAN',bg="#F8F9F9").place(relx=0.02,rely=0.8)
    Entry(p1,textvariable=rePAN).place(relx=0.25,rely=0.8,width=100)
    opt=IntVar()
    
    #---------------------------------------------------------------------------------------------------------------------------
    
    #----------------------------------------------------------------------------------------------------------------------------
    ttk.Separator(win,orient='horizontal').place(y=340,width=800,height=2)
    p3=PanedWindow(win,bg='#F8F9F9',height=200,width=760)
    p3.place(x=20,y=360)
    itms=StringVar()

    def mycommand(itms,p,par,idate,ino,veh,rname,raddr,rstate,rgst,rpan):
        for widget in p.winfo_children():
            widget.destroy()
        Label(p,text="Sno",bg="#F8F9F9").place(relx=0.02,rely=0.15)
        Label(p,text="Description",bg="#F8F9F9").place(relx=0.08,rely=0.15)
        Label(p,text="HSN/SAC",bg="#F8F9F9").place(relx=0.3,rely=0.15)
        Label(p,text="Qty (in cmt)",bg="#F8F9F9").place(relx=0.4,rely=0.15)
        Label(p,text="Rate",bg="#F8F9F9").place(relx=0.5,rely=0.15)
        Label(p,text="CGST",bg="#F8F9F9").place(relx=0.6,rely=0.15)
        Label(p,text="SGST",bg="#F8F9F9").place(relx=0.7,rely=0.15)
        Label(p,text="IGST",bg="#F8F9F9").place(relx=0.8,rely=0.15)
        Label(p,text="Total",bg="#F8F9F9").place(relx=0.9,rely=0.15)
        def calc(opts,qty,rate,e,x,p,idate,ino,veh,rname,raddr,rstate,rgst,rpan):
            hsn={"Teak Wood":"44","Imported Wood":"4407","Babule(Kukkar)":"4407","Firewood":"-"}
            idate=cal.get_date()
            cgst=[]
            sgst=[]
            igst=[]
            total=[]
            subtotal=[]
            for i in range(x):
                subtotal.append(round(float(qty[i].get())*float(rate[i].get()),2))
                if opts[i].get()!="Firewood":
                    if rstate.get()=="Punjab":
                        cgst.append(round(subtotal[i]*9/100,2))
                        sgst.append(round(subtotal[i]*9/100,2))
                        igst.append(0)
                    else:
                        cgst.append(0)
                        sgst.append(0)
                        igst.append(round(subtotal[i]*18/100,2))
                else:
                    cgst.append(0)
                    sgst.append(0)
                    igst.append(0)
                total.append(round(subtotal[i]+cgst[i]+sgst[i]+igst[i],2))
                Label(p,text="Rs."+str(cgst[i]),bg="#F8F9F9").place(relx=0.6,rely=0.15*(i+2))
                Label(p,text="Rs."+str(sgst[i]),bg="#F8F9F9").place(relx=0.7,rely=0.15*(i+2))
                Label(p,text="Rs."+str(igst[i]),bg="#F8F9F9").place(relx=0.8,rely=0.15*(i+2))
                Label(p,text="Rs."+str(total[i]),bg="#F8F9F9").place(relx=0.9,rely=0.15*(i+2))
                e[i].insert(0,hsn[opts[i].get()])
            grand=round(sum(total),2)
            Label(p,text="Grand Total: Rs."+str(grand),bg="#F8F9F9").place(relx=0.75,rely=0.90)
            conn=psycopg2.connect("dbname='InvoiceData' user='postgres' password='asdf1234' port='5432' host='localhost'")
            cur=conn.cursor()
            cur.execute(" INSERT INTO SellerInfo VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s)",(str(idate),str(veh.get()),str(ino.get()),str(rname.get()),str(raddr.get()),str(rstate.get()),str(rgst.get()),str(rpan.get()),str(x)))
            for i in range(x):
                cur.execute(" INSERT INTO SellerContent VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s)",(str(ino.get()),str(opts[i].get()),hsn[opts[i].get()],str(qty[i].get()),str(rate[i].get()),str(cgst[i]),str(sgst[i]),str(igst[i]),str(total[i])))
            cur.execute(" INSERT INTO BalanceInfo VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s)",(str(idate),str(veh.get()),str(ino.get()),str(rname.get()),str(raddr.get()),str(rstate.get()),str(rgst.get()),str(rpan.get()),str(x)))
            for i in range(x):
                cur.execute(" INSERT INTO BalanceContent VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s)",(str(ino.get()),str(opts[i].get()),hsn[opts[i].get()],str(qty[i].get()),str(rate[i].get()),str(cgst[i]),str(sgst[i]),str(igst[i]),str(total[i]))) 
            conn.commit()
            conn.close()
        x=int(itms.get())
        ops=["Teak Wood","Imported Wood","Babule(Kukkar)","Firewood"]
        qty=[]
        opts=[]
        e=[]
        rate=[]
        for i in range(x):
            opts.append(StringVar())
            qty.append(StringVar())
            rate.append(StringVar())

        for i in range(x):
            Label(p,text=str(i+1),bg="#F8F9F9").place(relx=0.02,rely=0.15*(i+2))
            OptionMenu( p , opts[i] , *ops).place(relx=0.08,rely=0.15*(i+2),height=25,width=150)
            e.append(Entry(p))
            e[i].place(relx=0.3,rely=0.15*(i+2),height=25,width=50)
            Entry(p,textvariable=qty[i]).place(relx=0.4,rely=0.15*(i+2),height=25,width=50)
            Entry(p,textvariable=rate[i]).place(relx=0.5,rely=0.15*(i+2),height=25,width=50)
        calc=partial(calc,opts,qty,rate,e,x,p,idate,ino,veh,rname,raddr,rstate,rgst,rpan)    
        Button(par,text="Calculate",command=calc).place(relx=0.7,rely=0.05,height=20,width=90)


    p4=PanedWindow(win,bg='#F8F9F9',height=190,width=760)
    p4.place(x=20,y=390)        
    mycommand=partial(mycommand,itms,p4,p3,InvoiceDate,Invoiceno,Vehicle,rename,reAddr,reState,reGST,rePAN)
    Label(p3,text='No of items',bg="#F8F9F9").place(relx=0.02,rely=0.05)
    Spinbox(p3,textvariable=itms,from_=1,to=4).place(relx=0.15,rely=0.05,width=100)
    Button(p3,text="Generate",command=mycommand).place(relx=0.32,rely=0.05,height=20,width=90)
    def srch(win):
        tp=Toplevel(win)
        tp.geometry("600x550")
        tp.resizable(0,0)
        Label(tp,text="Enter the Invoice No.").place(relx=0.1,rely=0.1)
        invno=StringVar()
        Entry(tp,textvariable=invno).place(relx=0.35,rely=0.1)
        def select(invno,pan):
            for widget in pan.winfo_children():
                widget.destroy()

            conn=psycopg2.connect("dbname='InvoiceData' user='postgres' password='asdf1234' port='5432' host='localhost'")
            cur=conn.cursor()
            cur.execute("SELECT * FROM SellerInfo WHERE InvoiceNo= (%s)",(str(invno.get()),))
            info=cur.fetchall()
            conn.commit()
            conn.close()
            if len(info)==0:
                Label(pan,text="No Info Found",bg="#F8F9F9",font="Arial 15").place(relx=0.02,rely=0.08)
            else:
                Label(pan,text="TAX INVOICE",bg='#F8F9F9',justify="center",font="Arial 15").place(relx=0.35,rely=0.09)
                ttk.Separator(pan,orient='horizontal').place(relx=0,rely=0.17,height=2,width=500)
                ttk.Separator(pan,orient='horizontal').place(relx=0,rely=0.6,height=2,width=500)
                s="Invoice Date: "+str(info[0][0])+"\tVehicle No: "+str(info[0][1])+"\tInvoice No: "+str(info[0][2])
                s1="Seller Information:\nName: "+str(info[0][3])+"\nAddress: "+str(" ".join(info[0][4].split(" ")[:4]))+"\n"+str(" ".join(info[0][4].split(" ")[4:8]))+"\n"+str(" ".join(info[0][4].split(" ")[8:]))+"\nState: "+str(info[0][5])+"\nGSTIN: "+str(info[0][6])+"\nPAN: "+str(info[0][7])
                Label(pan,text=s,bg='#F8F9F9',justify="left").place(relx=0.15,rely=0.2)
                Label(pan,text=s1,bg='#F8F9F9',justify="left").place(relx=0.15,rely=0.25)
                conn=psycopg2.connect("dbname='InvoiceData' user='postgres' password='asdf1234' port='5432' host='localhost'")
                cur=conn.cursor()
                cur.execute("SELECT * FROM SellerContent WHERE InvoiceNo= (%s)",(str(invno.get()),))
                info=cur.fetchall()
                conn.commit()
                conn.close()
                gtot=0.0
                Label(pan,text="Sno",bg="#F8F9F9").place(relx=0.02,rely=0.63)
                Label(pan,text="Description",bg="#F8F9F9").place(relx=0.08,rely=0.63)
                Label(pan,text="HSN",bg="#F8F9F9").place(relx=0.3,rely=0.63)
                Label(pan,text="Qty \n(in cmt)",bg="#F8F9F9").place(relx=0.39,rely=0.63)
                Label(pan,text="Rate",bg="#F8F9F9").place(relx=0.5,rely=0.63)
                Label(pan,text="CGST",bg="#F8F9F9").place(relx=0.57,rely=0.63)
                Label(pan,text="SGST",bg="#F8F9F9").place(relx=0.68,rely=0.63)
                Label(pan,text="IGST",bg="#F8F9F9").place(relx=0.8,rely=0.63)
                Label(pan,text="Total",bg="#F8F9F9").place(relx=0.89,rely=0.63)
                for i in range(len(info)):
                    Label(pan,text=str(i+1),bg="#F8F9F9").place(relx=0.02,rely=0.63+(0.05*(i+2)))
                    Label(pan,text=str(info[i][1]),bg="#F8F9F9").place(relx=0.08,rely=0.63+(0.05*(i+2)))
                    Label(pan,text=str(info[i][2]),bg="#F8F9F9").place(relx=0.3,rely=0.63+(0.05*(i+2)))
                    Label(pan,text=str(info[i][3]),bg="#F8F9F9").place(relx=0.39,rely=0.63+(0.05*(i+2)))
                    Label(pan,text=str(info[i][4]),bg="#F8F9F9").place(relx=0.5,rely=0.63+(0.05*(i+2)))
                    Label(pan,text=str(info[i][5]),bg="#F8F9F9").place(relx=0.57,rely=0.63+(0.05*(i+2)))
                    Label(pan,text=str(info[i][6]),bg="#F8F9F9").place(relx=0.68,rely=0.63+(0.05*(i+2)))
                    Label(pan,text=str(info[i][7]),bg="#F8F9F9").place(relx=0.8,rely=0.63+(0.05*(i+2)))
                    Label(pan,text=str(info[i][8]),bg="#F8F9F9").place(relx=0.89,rely=0.63+(0.05*(i+2)))
                    gtot=gtot+float(info[i][8])
                Label(pan,text="Grand Total: Rs."+str(gtot),bg="#F8F9F9").place(relx=0.7,rely=0.95)
                def delete(invno):
                    conn=psycopg2.connect("dbname='InvoiceData' user='postgres' password='asdf1234' port='5432' host='localhost'")
                    cur=conn.cursor()
                    cur.execute("DELETE FROM SellerContent WHERE InvoiceNo=(%s)",(invno.get(),))
                    cur.execute("DELETE FROM SellerInfo WHERE InvoiceNo=(%s)",(invno.get(),))
                    cur.execute("DELETE FROM BalanceContent WHERE InvoiceNo=(%s)",(invno.get(),))
                    cur.execute("DELETE FROM BalanceInfo WHERE InvoiceNo=(%s)",(invno.get(),))
                    conn.commit()
                    conn.close()
                delete=partial(delete,invno)    
                Button(pan,text="Delete",command=delete).place(relx=0.86,rely=0,height=20,width=70)
        pan=PanedWindow(tp,bg='#F8F9F9',height=400,width=500)
        pan.place(relx=0.08,rely=0.25)
        select=partial(select,invno,pan)
        Button(tp,text="Search",command=select).place(relx=0.7,rely=0.1,height=20,width=70)
    Button(win,text="Exit ",command=win.destroy).place(x=750,y=17)

    srch=partial(srch,win)
    Button(win,text="Search Invoice",command=srch).place(x=650,y=17)
    #l4=Label(win,text="Vehicle No")
    #l4.place(x=270,y=50)
    #e4=Entry(win,textvariable=GSTIN)
    #e4.place(x=350,y=50)


In [4]:
def balance(start):
    win=Toplevel(start,bg="#F8F9F9")
    win.geometry("600x500")
    win.resizable(0,0)
    Label(win,text="Select the Type of stock you want to view").place(relx=0.1,rely=0.1)
    ops=["Teak Wood","Imported Wood","Babule(Kukkar)","Firewood"]
    vw=StringVar()
    def view_data(vw,win):
        Label(win,text="Sno",bg="#F8F9F9").place(relx=0.02,rely=0.2)
        Label(win,text="Seller",bg="#F8F9F9").place(relx=0.18,rely=0.2)
        Label(win,text="Date",bg="#F8F9F9").place(relx=0.35,rely=0.2)
        Label(win,text="Remaining Qty \n(in cmt)",bg="#F8F9F9").place(relx=0.5,rely=0.2)
        Label(win,text="Rate",bg="#F8F9F9").place(relx=0.7,rely=0.2)
        Label(win,text="Amt",bg="#F8F9F9").place(relx=0.8,rely=0.2)
        conn=psycopg2.connect("dbname='InvoiceData' user='postgres' password='asdf1234' port='5432' host='localhost'")
        cur=conn.cursor()
        cur.execute("SELECT * FROM BalanceContent WHERE Description= (%s)",(str(vw.get()),))
        info1=cur.fetchall()
        for i in range(len(info1)):
            cur.execute("SELECT * FROM BalanceInfo WHERE InvoiceNo= (%s)",(str(info1[i][0]),))
            info=cur.fetchall()
            Label(win,text=str(i+1),bg="#F8F9F9").place(relx=0.02,rely=0.2+(0.05)*(i+2))
            Label(win,text=str(info[0][3]),bg="#F8F9F9").place(relx=0.08,rely=0.2+(0.05)*(i+2))
            Label(win,text=str(info[0][0]),bg="#F8F9F9").place(relx=0.35,rely=0.2+(0.05)*(i+2))
            Label(win,text=str(info1[i][3]),bg="#F8F9F9").place(relx=0.5,rely=0.2+(0.05)*(i+2))
            Label(win,text=str(info1[i][4]),bg="#F8F9F9").place(relx=0.7,rely=0.2+(0.05)*(i+2))
            Label(win,text=str(int(info1[i][4])*int(info1[i][3])),bg="#F8F9F9").place(relx=0.8,rely=0.2+(0.05)*(i+2))
        conn.commit()
        conn.close()
    OptionMenu( win , vw , *ops).place(relx=0.5,rely=0.1,height=25,width=150)
    view_data=partial(view_data,vw,win)
    Button(win,text="View",command=view_data).place(relx=0.76,rely=0.1,width=50)
    

In [5]:
start=Tk()
start.geometry("600x150")
start.resizable(0,0)
sale=partial(sale,start)
purchase=partial(purchase,start)
balance=partial(balance,start)
Label(start,text="Welcome Admin, Choose what you want to do:",font="Arial 12").place(relx=0.1, rely=0.1)
Button(start,text="Sales",command=sale).place(relx=0.15,rely=0.45,width=80)
Button(start,text="Purchase",command=purchase).place(relx=0.45,rely=0.45,width=80)
Button(start,text="Balance",command=balance).place(relx=0.75,rely=0.45,width=80)

start.mainloop()

SyntaxError: invalid syntax (<ipython-input-24-a10bfb847b84>, line 1)